Pydantic:
When you create an instance of a Pydantic model, it automatically parses and validates the input data.
It is widely used in FastAPI and other frameworks for clean, reliable API and data handling.
Pydantic also supports advanced features like custom validators, nested models, and JSON serialization.


In [1]:
from pydantic.dataclasses import dataclass

@dataclass
class User:
    id: int
    name: str
    age: int

# Automatically coerces and validates types
solution = User(id="1", name="Alice", age="30")
print(solution)

User(id=1, name='Alice', age=30)


BASE MODEL
Using Pydantic’s BaseModel is the most common way to define data models in Pydantic. It provides powerful features like: Automatic type, validation,Type coercion,Custom validators,Nested models

In [4]:
from pydantic import BaseModel, ValidationError
from typing import Optional,List


class Product(BaseModel):
    name: str
    price: float
    stock_list : Optional[List[str]] = []
    in_stock: Optional[bool] = False  # Now it's optional

item = Product(name="Chocolate", price='2.99',stock_list=["item1", "item2"])
item1 = Product(name="Chocolate", price=2.99,in_stock=True,stock_list=("item1", "item2",1))
print(item)
print(item1)

ValidationError: 1 validation error for Product
stock_list.2
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

STRICT MODE:
--------------------
Float allows type coercion, accepting strings, ints, and bools (e.g., “3.14”, 5, True → float).
StrictFloat enforces exact float type, rejecting any non-float input (e.g., “3.14” or 5 will raise an error).

In [5]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str
    price: float = Field(strict=True, description="Price of the product")

item = Product(name="Chocolate", price=2.99)
item1 = Product(name="Chocolate", price="2")

print(item)
print(item1)

ValidationError: 1 validation error for Product
price
  Input should be a valid number [type=float_type, input_value='2', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_type

Nested Models in Pydantic:
--------------------------------
Pydantic supports nested models, where one BaseModel is used as a field inside another.
It automatically validates and parses nested dictionaries into model instances.

In [6]:
from pydantic import BaseModel
from typing import Optional

class Category(BaseModel):
    id: int
    name: str

class Product(BaseModel):
    name: str
    price: float
    in_stock: Optional[bool] = True
    category: Category  # Nested model

# Creating a product with a nested category
item = Product(
    name="Chocolate",
    price=2.99,
    category=Category(id=1, name="Snacks")
)

print(item)

name='Chocolate' price=2.99 in_stock=True category=Category(id=1, name='Snacks')


***Field*** is used to add metadata, defaults, validation constraints, and documentation to model fields.

In [7]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str = Field(..., min_length=2, max_length=50, description="Product name")
    price: float = Field(..., gt=0, description="Must be greater than 0")
    stock: int = Field(default=0, ge=0, description="Non-negative integer")

product = Product(name="Apple", price=1.25)
print(product)


name='Apple' price=1.25 stock=0


***NUMERIC CONSTRAINTS***
There are some keyword arguments that can be used to constrain numeric values:

gt - greater than
lt - less than
ge - greater than or equal to
le - less than or equal to
multiple_of - a multiple of the given number
allow_inf_nan - allow 'inf', '-inf', 'nan' values

In [8]:
from pydantic import BaseModel, Field


class Foo(BaseModel):
    positive: int = Field(gt=0)
    non_negative: int = Field(ge=0)
    negative: int = Field(lt=0)
    non_positive: int = Field(le=0)
    even: int = Field(multiple_of=2)
    love_for_pydantic: float = Field(allow_inf_nan=True)


foo = Foo(
    positive=1,
    non_negative=0,
    negative=-1,
    non_positive=0,
    even=2,
    love_for_pydantic=float('inf'),
)
print(foo)

positive=1 non_negative=0 negative=-1 non_positive=0 even=2 love_for_pydantic=inf


STRING CONSTRAINTS
There are fields that can be used to constrain strings:
min_length: Minimum length of the string.
max_length: Maximum length of the string.
pattern: A regular expression that the string must match.

In [11]:
from pydantic import BaseModel, Field


class Foo(BaseModel):
    short: str = Field(min_length=3)
    long: str = Field(max_length=10)
    regex: str = Field(pattern=r'^\d*$')  


foo = Foo(short='foo', long='foobarbaz', regex='123')
print(foo)

short='foo' long='foobarbaz' regex='123'


@field_validator
@field_validator is used to define custom validation logic for specific fields in a Pydantic model.

In [12]:
from pydantic import BaseModel, field_validator

class User(BaseModel):
    age: int

    @field_validator('age')
    def age_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError('Age must be positive')
        return v
user_0 = User(age=25)
print(user_0)
user_1 = User(age=-25)
print(user_1)

age=25


ValidationError: 1 validation error for User
age
  Value error, Age must be positive [type=value_error, input_value=-25, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error